# Clamped Spline

En este notebook se presenta, programado en Python, el algoritmo de cubic spline utilizado por CXVA para la interpolación de curvas cero cupón.

Carguemos la curva cero cupón a utilizar como ejemplo:

In [1]:
# Se importa el módulo pandas
import pandas as pd

# Se carga el csv en el DataFrame curva
curva = pd.read_csv("./curva_2.csv")

# Se muestra toda la data. Los plazos están en años y la curva está expresada
# en términos de factores de descuento (df).
curva                                   

,Plazo,Df
0,0.002740,0.999917
1,0.005479,0.999833
2,0.254795,0.992906
3,0.506849,0.986198
4,0.758904,0.979555
5,1.005479,0.973384
6,1.506849,0.959941
7,2.005479,0.944978
8,3.010959,0.912039
9,4.008219,0.877304


Agregamos una columna a **curva** con la tasa en composición compuesta contínua. Es decir $df=\mathrm{e}^{-rt}$, por lo tanto, $r=-\frac{\mathrm{log}(Df)}{t}$. Para hacerlo rápidamente vamos a importar **numpy**, librería que nos permite entre otras cosas, aplicar funciones a arreglos (como la columna Df de **curva**).

In [2]:
# Se importa el módulo numpy con el nombre breve np.
import numpy as np

# Se calculan las tasas a partir de los factores de descuento
# y se agregan al DataFrame anterior.
curva['Tasa'] = -np.log(curva['Df'])/curva['Plazo']    

# Se imprime el DataFrame curva con la nueva columna.
curva                                                  

,Plazo,Df,Tasa
0,0.002740,0.999917,0.030415
1,0.005479,0.999833,0.030415
2,0.254795,0.992906,0.027941
3,0.506849,0.986198,0.027422
4,0.758904,0.979555,0.027219
5,1.005479,0.973384,0.026829
6,1.506849,0.959941,0.027131
7,2.005479,0.944978,0.028220
8,3.010959,0.912039,0.030579
9,4.008219,0.877304,0.032658


Se define ahora una función que calcula los coeficientes asociados a un **Clamped Cubic Spline**. Los valores de las abscisas serán los elementos del vector **Plazo** de la curva y las ordenadas serán los elementos del vector **Tasa**.

In [20]:
def get_clamped_spline_coef(x, y):
    if len(x) != len(y):
        return "The sizes of the input ranges are different"

    # Los parámetros x e y se transforman en numpy arrays.
    x = np.array(x)
    y = np.array(y)
    
    n = len(x) - 1;

    # Se definen las variables que se retornan
    a = np.zeros(n + 1)
    b = np.zeros(n + 1)
    c = np.zeros(n + 1)
    d = np.zeros(n + 1)

    # Se comienza
    fp1 = 0.0
    fpn = 0.0
    # Modificar vectores a, b, c
    # vector <double> h(n), l(n+1), u(n+1);
    h = np.zeros(n)
    l = np.zeros(n + 1)
    u = np.zeros(n + 1)
    
    # vector <double> alpha(n+1), z(n+1);
    alpha = np.zeros(n + 1)
    z = np.zeros(n + 1)

    fp1 = (y[1] - y[0]) / (x[1] - x[0])
    fpn = (y[n] - y[n - 1]) / (x[n] - x[n - 1])
  
    # for (unsigned int i=0; i < n; i++)
        # h.at(i) = x[i + 1] - x[i];
    for i in range(0, n):
        h[i] = x[i + 1] - x[i]
        
    alpha[0] = ((y[1] - y[0]) * 3) / h[0] - fp1 * 3
    alpha[n] = fpn * 3 - ((y[n] - y[n - 1]) * 3) / h[n - 1]

    # for (unsigned int i=1; i < n ; i++)
    for i in range(1, n):
        alpha[i] = ((y[i + 1] - y[i])) * (3.0 / h[i]) - ((y[i] - y[i - 1])) * (3.0 / h[i - 1])                                  
        
    l[0] = 2 * h[0]
    u[0] = 0.5
    z[0] = alpha[0] / l[0]

    # for (unsigned int i=1; i < n ; i++)
    for i in range(1, n):
        l[i] = 2 * (x[i + 1] - x[i - 1]) - h[i - 1] * u[i - 1]
        u[i] = h[i] / l[i]
        z[i] = (alpha[i] - z[i - 1] * h[i - 1]) / l[i]

    l[n] = h[n - 1] * (2 - u[n - 1])
    z[n] = (alpha[n] - z[n - 1] * h[n - 1]) / l[n]
    c[n] = z[n]
        
    # for (j=1; j < (n+1) ; j++) 
    for j in range(1, n + 1):
        i = n - j
        c[i] = z[i] - c[i + 1] * u[i]
        b[i] = (y[i + 1] - y[i]) / h[i] - ((c[i + 1] + c[i] * 2) * h[i])/ 3
        d[i] = (c[i + 1] - c[i]) /(3 *h[i])
        a[i] = y[i]
    
    b[n] = d[n] = 0
    a[n] = y[n]
    
    return a, b, c, d                     

Se prueba la función:

In [41]:
# Se define un np.array con los plazos
plazos = np.array(curva['Plazo'].tolist())
print("Plazos", plazos)

 # Se define un np.array con las tasas
tasas = np.array(curva['Tasa'].tolist())
print("Tasas:", tasas)

# Se calculan los coeficientes
result = get_clamped_spline_coef(plazos, tasas)

for i in range(0, 4):
    print(result[i])

('Plazos', array([  2.73972603e-03,   5.47945205e-03,   2.54794521e-01,
         5.06849315e-01,   7.58904110e-01,   1.00547945e+00,
         1.50684932e+00,   2.00547945e+00,   3.01095890e+00,
         4.00821918e+00,   5.00821918e+00,   6.00821918e+00,
         7.01095890e+00,   8.01643836e+00,   9.01369863e+00,
         1.00109589e+01,   1.20136986e+01,   1.50164384e+01,
         2.00219178e+01]))
('Tasas:', array([ 0.0304154 ,  0.0304154 ,  0.02794056,  0.02742158,  0.0272191 ,
        0.02682926,  0.02713148,  0.02821959,  0.03057911,  0.03265835,
        0.0344492 ,  0.0360535 ,  0.03780464,  0.03912898,  0.03988895,
        0.04054917,  0.04142247,  0.04217662,  0.04324746]))
[ 0.0304154   0.0304154   0.02794056  0.02742158  0.0272191   0.02682926
  0.02713148  0.02821959  0.03057911  0.03265835  0.0344492   0.0360535
  0.03780464  0.03912898  0.03988895  0.04054917  0.04142247  0.04217662
  0.04324746]
[ -7.30125682e-11  -1.12220375e-04  -9.13110296e-03   5.54674681e-04
  -1.67

Con esta función para encontrar coeficientes se puede generar una nueva función que utilice el spline para interpolar en la curva. Adicionalmente, se retornará el valor de la derivada primera y de la derivada segunda.

In [66]:
def get_clamped_spline_interpol(plazos, tasas):
    result = get_clamped_spline_coef(plazos, tasas)
    def cs(t):
        ind = np.where(plazos<=t)
        if len(ind[0]) == 0:
            index = 0
        else:
            index = np.max(ind)
        t -= plazos[index]
        valor = result[0][index] + result[1][index] * t + result[1][index] * t**2 + result[2][index] * t**3
        der = result[1][index] + 2 * result[1][index] * t + 3 * result[2][index] * t**2
        der2 = 2 * result[1][index] + 6 * result[2][index] * t
        return valor, der, der2
    return cs

In [74]:
cspline = get_clamped_spline_interpol(plazos, tasas)
cspline(0)[0]

0.030415398534012462